In [2]:
from pathlib import Path
import pandas as pd

data_path = Path(r"C:\Users\wojte\Desktop\Hacknation2025\data\df_pivot_filtered_numbers.csv")
df = pd.read_csv(data_path)

def add_pct_change(df: pd.DataFrame, metric: str) -> None:
    suffix = f'_{metric}'
    base_cols = [c for c in df.columns if c.endswith(suffix) and '_to_' not in c and not c.endswith('_to')]
    for col in base_cols:
        base = col[:-len(suffix)]
        dyn_col = f'{base}_{metric}_dyn'
        df[dyn_col] = df[col].pct_change()

for metric in ['GS', 'NP', 'CF']:
    add_pct_change(df, metric)

gs_cols = {c[:-3]: c for c in df.columns if c.endswith('_GS') and '_to_' not in c and not c.endswith('_to')}
ltl_cols = {c[:-4]: c for c in df.columns if c.endswith('_LTL') and '_to_' not in c and not c.endswith('_to')}
stl_cols = {c[:-4]: c for c in df.columns if c.endswith('_STL') and '_to_' not in c and not c.endswith('_to')}
common_keys = sorted(gs_cols.keys() & ltl_cols.keys() & stl_cols.keys())
for key in common_keys:
    debt_col = f'{key}_debt_to_revenue'
    df[debt_col] = (df[ltl_cols[key]] + df[stl_cols[key]]) / df[gs_cols[key]]

df.to_csv(data_path, index=False)
print(f'Updated {len(df.columns)} columns; added debt_to_revenue for {len(common_keys)} groups.')

Updated 1126 columns; added debt_to_revenue for 15 groups.


C:\Users\wojte\AppData\Local\Temp\ipykernel_29860\4035375652.py:13: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[dyn_col] = df[col].pct_change()


In [3]:
import re

df_ratios = df.copy()  # replace df with your DataFrame variable

# identify numeric prefixes like "10.1", "22.2", …
prefixes = sorted({c.split("_", 1)[0] for c in df_ratios.columns if re.match(r"^\d", c)})

keep_cols = ["rok"]
for p in prefixes:
    candidates = [
        f"{p}_GS_dyn",
        f"{p}_NP_dyn",
        f"{p}_CF_dyn",
        f"{p}_DEBT_to_GS",
        f"{p}_CF_to_TC",
        f"{p}_STL_to_GS",
    ]
    # add ratio columns if source data exists
    np_col, gs_col, tc_col = f"{p}_NP", f"{p}_GS", f"{p}_TC"
    if np_col in df_ratios.columns and gs_col in df_ratios.columns:
        ratio_col = f"{p}_NP_to_GS"
        df_ratios[ratio_col] = df_ratios[np_col] / df_ratios[gs_col]
        candidates.append(ratio_col)
    if np_col in df_ratios.columns and tc_col in df_ratios.columns:
        ratio_col = f"{p}_NP_to_TC"
        df_ratios[ratio_col] = df_ratios[np_col] / df_ratios[tc_col]
        candidates.append(ratio_col)

    keep_cols.extend(c for c in candidates if c in df_ratios.columns)

df_slice = df_ratios[keep_cols].copy()
df_slice.head()


,rok,10.1_GS_dyn,10.1_NP_dyn,10.1_CF_dyn,10.1_DEBT_to_GS,10.1_CF_to_TC,10.1_STL_to_GS,10.1_NP_to_GS,10.1_NP_to_TC,22.2_GS_dyn,...,52.2_NP_to_GS,52.2_NP_to_TC,62.0_GS_dyn,62.0_NP_dyn,62.0_CF_dyn,62.0_DEBT_to_GS,62.0_CF_to_TC,62.0_STL_to_GS,62.0_NP_to_GS,62.0_NP_to_TC
0,2005,NaN,NaN,NaN,0.200806,0.048337,0.133217,0.019857,0.020330,NaN,...,0.050816,0.054346,NaN,NaN,NaN,0.344450,0.142564,0.290866,0.075210,0.082341
1,2006,0.042270,-0.019871,0.042457,0.208385,0.048288,0.141798,0.018673,0.019096,0.140223,...,0.056970,0.061527,-0.009348,-0.089460,-0.030471,0.323346,0.139568,0.277943,0.069128,0.075706
2,2007,0.122495,0.227003,0.151375,0.202369,0.049630,0.146417,0.020411,0.020915,0.187502,...,0.072333,0.079331,0.146006,0.209094,0.068826,0.336678,0.130897,0.271881,0.072933,0.080321
3,2008,0.044975,-0.635139,-0.047439,0.199708,0.044524,0.146968,0.007127,0.007187,0.004993,...,0.029818,0.030944,0.328905,0.578132,0.543354,0.282956,0.154318,0.237092,0.086612,0.096826
4,2009,0.091467,0.724386,0.164312,0.176530,0.047746,0.130255,0.011259,0.011415,0.043485,...,0.043434,0.045920,0.057097,0.071153,0.008324,0.259020,0.147766,0.215997,0.087763,0.098492


In [4]:
print(df_slice)

     rok  10.1_GS_dyn  10.1_NP_dyn  10.1_CF_dyn  10.1_DEBT_to_GS  \
0   2005          NaN          NaN          NaN         0.200806   
1   2006     0.042270    -0.019871     0.042457         0.208385   
2   2007     0.122495     0.227003     0.151375         0.202369   
3   2008     0.044975    -0.635139    -0.047439         0.199708   
4   2009     0.091467     0.724386     0.164312         0.176530   
5   2010     0.009726     1.394732     0.130790         0.180000   
6   2011     0.175451    -0.292150    -0.095271         0.179125   
7   2012     0.136095     0.194771     0.135127         0.169286   
8   2013     0.019817     0.092016     0.016090         0.169305   
9   2014     0.008091     0.575780     0.261256         0.172069   
10  2015     0.010875    -0.009815     0.036379         0.188268   
11  2016     0.076187    -0.130738    -0.042158         0.195434   
12  2017     0.086806     0.144680     0.129474         0.200487   
13  2018    -0.000597     0.284304     0.163967 

In [5]:
df_slice.to_csv(data_path.parent / "df_ratios_finished_final_8_ratios_jig.csv", index=False)